# Word2Vec

## Import Library

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import joblib

from sentence_transformers import SentenceTransformer

2023-06-25 10:19:18.031683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 10:19:18.552558: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-25 10:19:18.602119: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-25 10:19:18.602188: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

## Import Model Embedding

In [3]:
embedding_nnlm = tf.keras.models.load_model("../../model/nnlm-id-dim50/")
embedding_indobert = SentenceTransformer("../../model/indobert-base-uncased")

2023-06-25 10:19:32.861656: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-25 10:19:32.861755: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-25 10:19:32.861786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SAT-PC): /proc/driver/nvidia/version does not exist
2023-06-25 10:19:32.862307: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
No sentence-transformers model found with name ../../mode

In [25]:
def embed_text_nnlm(model : object, 
               text : str) -> list :
    """Function to create embedding 
        using model object

    Args:
        model (object): Embedding model
        text (str): text input

    Returns:
        list: embedding of the text input
    """
    try :
        embedding_list = model([text]).numpy().squeeze().tolist()
        return embedding_list
    except : 
        print(f"The error text is : {text}")
        return None
    

## Vetorize Data

Data Source : 
- https://github.com/kirralabs/indonesian-NLP-resources
- https://github.com/sastrawi/sastrawi/blob/master/data/kata-dasar.original.txt
- https://github.com/edwardsamuel/Wilayah-Administratif-Indonesia/tree/master/csv

In [26]:
# import data
district_data = pd.read_csv("../../data/raw/districts.csv", header=None)
province_data = pd.read_csv("../../data/raw/provinces.csv", header=None)
regencies_data = pd.read_csv("../../data/raw/regencies.csv", header=None)
villages_data = pd.read_csv("../../data/raw/villages.csv", header=None)
kata_data = pd.read_table("../../data/raw/kata-dasar.original.txt", header=None)

# format data
district_data = district_data[2].apply(lambda x: x.lower())
province_data = province_data[1].apply(lambda x: x.lower())
regencies_data = regencies_data[2].apply(lambda x: x.lower())
villages_data = villages_data[2].apply(lambda x: x.lower())

# concat data
dataset = pd.concat([district_data, province_data, regencies_data, villages_data, kata_data], axis=0)
dataset.columns = ["words"]
dataset.drop_duplicates(inplace=True)
dataset.dropna(inplace=True)
dataset.reset_index(drop=True, inplace=True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93566 entries, 0 to 93565
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   words   93566 non-null  object
dtypes: object(1)
memory usage: 731.1+ KB


In [27]:
# create embedding nnlm
dataset_nnlm = dataset.copy()
dataset_nnlm['embedding'] = dataset_nnlm['words'].apply(lambda x : embed_text_nnlm(model=embedding_nnlm, text=x))
joblib.dump(dataset_nnlm,"../../data/processed/word_embedding_nnlm.pkl")

The error text is : kupan jaya


In [14]:
dataset_nnlm = joblib.load("../../data/processed/word_embedding_nnlm.pkl")

## create train and test data
X_nnlm = dataset_nnlm['embedding']
y_nnlm = dataset_nnlm['words']

## format data train and numpy array
X_nnlm =  np.array(X_nnlm.to_numpy().tolist())
y_nnlm = y_nnlm.values.astype('str')

# dumpy daata into X_nnlm.pkl and y_nnlm.pkl
joblib.dump(X_nnlm, "../../data/processed/X_nnlm.pkl")
joblib.dump(y_nnlm, "../../data/processed/y_nnlm.pkl")

['../../data/processed/y_nnlm.pkl']

In [7]:
# create embedding indobert
dataset_indobert = kata_data.copy()
dataset_indobert.columns = ['words']
dataset_indobert.drop_duplicates(inplace=True)
dataset_indobert.dropna(inplace=True)
dataset_indobert.reset_index(drop=True, inplace=True)
dataset_indobert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30342 entries, 0 to 30341
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   words   30341 non-null  object
dtypes: object(1)
memory usage: 237.2+ KB


In [8]:
def embed_indobert(model, text) :
    try :
        return model.encode(text).tolist()
    except :
        print(f"Text causing error : {text}")
        return None

In [9]:
dataset_indobert['embedding'] = dataset_indobert['words'].apply(lambda x : embed_indobert(model=embedding_indobert, text=x))
joblib.dump(dataset_indobert, "../../data/processed/word_embedding_indobert.pkl")

Text causing error : nan


['../../data/processed/word_embedding_indobert.pkl']